In [1]:
import pandas as pd
import os
import re
import openpyxl

In [2]:
print(os.getcwd())

/Users/notvisal/Desktop/InternProj/knife_sharpness


In [3]:
P1_contents = os.listdir('Theme2/P1/Boning/')

print(P1_contents)

['MVN-J-Boning-79-001.xlsx', 'MVN-J-Boning-64-001.xlsx', 'MVN-J-Boning-64-006.xlsx', 'MVN-J-Boning-90-001.xlsx', 'MVN-J-Boning-64-004.xlsx', 'MVN-J-Boning-90-002.xlsx', 'MVN-J-Boning-90-003.xlsx', 'MVN-J-Boning-64-005.xlsx', 'MVN-J-Boning-90-004.xlsx', 'MVN-J-Boning-64-002.xlsx', 'MVN-J-Boning-64-003.xlsx']


In [17]:
P1_BONING_PATHS = [
    'Theme2/P1/Boning/MVN-J-Boning-64-001.xlsx', 
    'Theme2/P1/Boning/MVN-J-Boning-90-003.xlsx', 
    'Theme2/P1/Boning/MVN-J-Boning-90-002.xlsx', 
    'Theme2/P1/Boning/MVN-J-Boning-90-004.xlsx', 
    'Theme2/P1/Boning/MVN-J-Boning-64-006.xlsx', 
    'Theme2/P1/Boning/MVN-J-Boning-64-004.xlsx', 
    'Theme2/P1/Boning/MVN-J-Boning-64-002.xlsx', 
    'Theme2/P1/Boning/MVN-J-Boning-90-001.xlsx', 
    'Theme2/P1/Boning/MVN-J-Boning-64-003.xlsx', 
    'Theme2/P1/Boning/MVN-J-Boning-79-001.xlsx', 
    'Theme2/P1/Boning/MVN-J-Boning-64-005.xlsx'
]

ACCELERATION_SHEETS = [
    'Segment Acceleration', 
    'Segment Angular Acceleration'
]

In [18]:
xls_file = pd.ExcelFile(P1_BONING_PATHS[1])

print(xls_file.sheet_names)

df = xls_file.parse('Segment Acceleration')
df.head()

['General Information', 'Markers', 'Segment Orientation - Quat', 'Segment Orientation - Euler', 'Segment Position', 'Segment Velocity', 'Segment Acceleration', 'Segment Angular Velocity', 'Segment Angular Acceleration', 'Joint Angles ZXY', 'Joint Angles XZY', 'Ergonomic Joint Angles ZXY', 'Ergonomic Joint Angles XZY', 'Center of Mass', 'Sensor Free Acceleration', 'Sensor Magnetic Field', 'Sensor Orientation - Quat', 'Sensor Orientation - Euler']


,Frame,Pelvis x,Pelvis y,Pelvis z,L5 x,L5 y,L5 z,L3 x,L3 y,L3 z,...,Left Upper Leg z,Left Lower Leg x,Left Lower Leg y,Left Lower Leg z,Left Foot x,Left Foot y,Left Foot z,Left Toe x,Left Toe y,Left Toe z
0,0,0,0,0,-0.309156,0.166350,-0.005329,-0.446559,0.240283,-0.007697,...,0.086429,-0.361156,0.124165,0.809022,-0.206274,-0.454396,0.744646,-0.206274,-0.454396,0.744646
1,1,0,0,0,-0.211907,0.045369,-0.073020,-0.308534,0.061313,-0.105288,...,-0.061544,-0.145087,-0.635056,0.745364,-0.142400,-0.781116,0.532242,-0.124329,-0.762226,0.532774
2,2,0,0,0,0.064328,0.604437,0.223626,0.090622,0.869357,0.323181,...,0.020390,-0.020240,-0.607848,0.592799,-0.303746,-0.549254,0.624062,-0.285954,-0.528629,0.624407
3,3,0,0,0,-0.252743,0.177120,-0.164212,-0.358088,0.267682,-0.241593,...,-0.005595,-0.065186,-0.411955,0.595625,-0.337684,-0.159395,0.581226,-0.361522,-0.239287,0.662329
4,4,0,0,0,-0.154837,-0.149689,-0.113165,-0.228697,-0.219128,-0.163050,...,-0.150496,-0.371449,-0.606982,0.684114,-0.224017,-0.330604,0.401518,-0.207623,-0.311683,0.401845


In [19]:
len(df.columns.tolist())

70

# **Merging**

In [27]:
from collections import defaultdict

sharpness_levels = [64, 79, 90]

def process_acceleration_data(file_paths, sharpness_levels):
    """Process acceleration data from multiple Excel files with different sharpness levels."""
    # Dictionary to store processed data by sheet type
    sheet_data = defaultdict(list)
    
    # Process all files and extract data by sheet type
    for file_path, sharpness in zip(file_paths, sharpness_levels):
        try:
            xls = pd.ExcelFile(file_path)
            
            for sheet_name in xls.sheet_names:
                if sheet_name not in ACCELERATION_SHEETS:
                    continue
                
                try:
                    # Read the sheet
                    df = pd.read_excel(xls, sheet_name=sheet_name)
                    print(f"{df.shape}")
                    if df.empty:
                        continue
                    
                    # Add sharpness column
                    df['sharpness'] = sharpness
                    
                    # Add sheet name as a column to differentiate data source
                    df['sheet_type'] = sheet_name
                    
                    # Add to our collection
                    sheet_data[sheet_name].append(df)
                
                except Exception as e:
                    print(f"Error processing sheet {sheet_name}: {e}")
        
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
    
    # Combine data from all files
    combined_dfs = []
    
    for sheet_name, dfs in sheet_data.items():
        if dfs:
            # Concatenate all data for this sheet type
            sheet_combined_df = pd.concat(dfs, ignore_index=True)
            combined_dfs.append(sheet_combined_df)
    
    # Return combined data frame
    if combined_dfs:
        final_df = pd.concat(combined_dfs, ignore_index=True)
        return final_df
    
    return None

def main():
    # Process the data
    merged_df = process_acceleration_data(
        file_paths=P1_BONING_PATHS,
        sharpness_levels=sharpness_levels
    )
    
    if merged_df is None or merged_df.empty:
        print("No results generated!")
        return
    
    # Save the final merged dataframe
    output_file = "acceleration_data.csv"
    merged_df.to_csv(output_file, index=False)
    print(f"Saved data to {output_file} ({merged_df.shape[0]} rows, {merged_df.shape[1]} columns)")

if __name__ == "__main__":
    main()

(2957, 71)
(2957, 71)
(15758, 70)
(15758, 71)
(22841, 71)
(22841, 71)
Saved data to acceleration_data.csv (83112 rows, 73 columns)


In [28]:
df = pd.read_csv('acceleration_data.csv')

df.head()

,Frame,Label,Pelvis x,Pelvis y,Pelvis z,L5 x,L5 y,L5 z,L3 x,L3 y,...,Left Lower Leg y,Left Lower Leg z,Left Foot x,Left Foot y,Left Foot z,Left Toe x,Left Toe y,Left Toe z,sharpness,sheet_type
0,0,4.0,0.0,0.0,0.0,-1.640562,-0.345798,-0.666146,-2.369700,-0.499486,...,-2.060481,-1.485525,-2.185359,-1.194785,-1.663235,-2.185359,-1.194785,-1.663235,64,Segment Acceleration
1,1,4.0,0.0,0.0,0.0,-0.705326,-0.629461,-0.983502,-1.043948,-0.907626,...,-2.689504,-1.314239,-1.060466,-1.162565,-1.845591,-1.064713,-1.147108,-1.845636,64,Segment Acceleration
2,2,4.0,0.0,0.0,0.0,-1.146854,-0.839369,-0.862127,-1.683033,-1.219342,...,-2.178955,-1.321744,-0.220440,-1.530509,-1.658241,-0.227187,-1.514522,-1.657987,64,Segment Acceleration
3,3,4.0,0.0,0.0,0.0,-0.065640,-0.431009,-0.488192,-0.426137,-0.574987,...,-0.951495,-1.324380,-0.175790,-1.007643,-1.326566,-0.091278,-0.986575,-1.316862,64,Segment Acceleration
4,4,4.0,0.0,0.0,0.0,0.652927,-0.046127,0.174043,0.920047,-0.080622,...,-0.402592,-0.509365,0.347853,-0.509991,-0.538392,0.335083,-0.493518,-0.537629,64,Segment Acceleration


In [29]:
df.isna().sum()

Frame             0
Label         15758
Pelvis x          0
Pelvis y          0
Pelvis z          0
              ...  
Left Toe x        0
Left Toe y        0
Left Toe z        0
sharpness         0
sheet_type        0
Length: 73, dtype: int64